In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from ripser import ripser
from persim import plot_diagrams
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout 
import tensorflow as tf
import random as rn

Using TensorFlow backend.


In [6]:
def assign_label(img,cancer_type):
    return cancer_type

In [7]:
def make_train_data(cancer_type,DIR):
    for img in os.listdir(DIR):
        label=assign_label(img,cancer_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            
        X.append(np.array(img))
        Z.append(str(label))

In [8]:
X=[]
Z=[]
IMG_SIZE=50
bc_malignant_DIR='/Users/user/Desktop/breast_cancer/malignant'
bc_benign_DIR='/Users/user/Desktop/breast_cancer/benign'

In [9]:
print(os.listdir('/Users/user/Desktop/breast_cancer'))
make_train_data('benign',bc_benign_DIR)
print(len(X))
make_train_data('malignant',bc_malignant_DIR)
print(len(X))

['benign', 'malignant']
16
31


In [10]:
le=LabelEncoder()
Y=le.fit_transform(Z)
s=set(Z)
l=[]
for i in s:
    l.append(i)
cheak1=le.fit_transform(l)
cheak2=to_categorical(cheak1,2)
Y=to_categorical(Y,2)
X=np.array(X)
X=X/255

In [73]:
from skimage import color
from skimage import io

In [74]:
X1=[]
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
DIR1='/Users/user/Desktop/breast_cancer/benign'
DIR2='/Users/user/Desktop/breast_cancer/malignant'
for img in os.listdir(DIR1):
        path = os.path.join(DIR1,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = color.rgb2gray(img)
        ret, b_img = cv2.threshold(img,.5,1,cv2.THRESH_BINARY)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        X1.append(b_img)
for img in os.listdir(DIR2):
        path = os.path.join(DIR2,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = color.rgb2gray(img)
        ret, b_img = cv2.threshold(img,.5,1,cv2.THRESH_BINARY)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        X1.append(b_img)

In [75]:
X1=np.array(X1)
print(X1.shape)

(31, 50, 50)


In [76]:
X3=[]
for i in range(31):
    X2=[]
    for j in range(50):
        for k in range(50):
            X2.append(X1[i][j][k])
            
    X3.append(X2)

In [77]:
X4=X3[:22]
X5=X3[-10:] 
Y1=Y[:22]
Y2=Y[-10:]

In [78]:

X3=np.array(X3)
print(X3.shape)

(31, 2500)


In [79]:

# Function to return the betti numbers for dimensions 0 and 1, of the simplicial complex
# created form a Rips filtration of a point cloud.
def betti_nums(data, sType, scanStart, scanStop, eps = 100, showPlot = False, doSlice = False):
    '''Inputs: -data: Your data set. Should be a list of 1's and 0's.
    
               -Stype: Scanning type. Should be a string either, 'ud'(up-and-down),
                       'lr'(left-to-right), or 'rl'(right-to-left). 
                       *For down-and-up scanning, fix scanStop at the last index of 
                        your list, and alter scanStart.*
                       **For slices, choose 'ud' for horizontal slices. Then choose scanStart 
                         and scanStop to match where you want to start and end your slice.**
                       ***For middle-out scanning, repeat the slicing procedure, but
                          increase scanStart and scanStop at the same time to gradually
                          increment the size of the slice.***
                       
               -scanStart: Where to start scanning. For 'ud' and 'lr' scanning
                           fix this to be the first index.
                
               -scanStop: Where to stop scanning. For 'du' scanning
                          fix this to be the last index.
                          
               -eps: The max distance in a Rips filtration. This becomes our, "infinity."
                     This is initialized to 100. For more precision, decrease this to fit
                     your data set.
                     
               -showPlot: Whether or not to show the PH diagrams. Initialized to False, change
                          to True to show PH diagrams.
                
               -doSlice: Whether or not to so slice scanning. Initialized to False. Set to
                         True if you wish to do slice scanning.
        
       Outputs: -(b0, b1): A tuple containing betti0 and betti1 for the scanning data set,
                           depending on the scanning type and how much was scanned.
    '''
    
    letter = np.array([[0,0]])    # Initializes an array of (1x2)-arrays.
    
    # Up-and-Down scanning.
    if sType.lower() == 'ud':
        
        # Loops through the line with the letter data and if the kth position
        # of the array is a 1, then the coordinates of that 1 is added as a
        # 1x2 array to the letter array.
        for k in range(scanStart,scanStop):
            if data[k] == 1:
                col = 28-int((k-1)/28)
                row = (k-1)%28
                letter = np.append(letter,[[row, col]], axis = 0)
        
        # Removes the first entry of letter, as it was a placeholder.
        letter = np.delete(letter, 0, 0)
        
        # Test for the space (or just nothing), otherwise, the function breaks.
        if len(letter) == 0:
            return (0,0)
        
        # Filters the point cloud, using a Rips filtration up to a max dimension of 2.
        diagrams = ripser(letter, maxdim = 1, metric = 'euclidean',thresh = eps)['dgms']
        
        # If the you want to see the shape of the data and the persistence diagram
        # both are displayed.
        if showPlot == True:
            # Setting the x and y coordinates to be plotted to show the shape
            # of the data.
            x_data = [letter[i][0] for i in range(0,len(letter))]
            y_data = [letter[i][1] for i in range(0,len(letter))]
        
            # Plots the shape of the data
            plt.figure(1)
            plt.plot(x_data, y_data, 'ro', scalex = True, scaley = True)
            axes = plt.gca()
            axes.set_xlim([-1,29])
            axes.set_ylim([-1,29])
        
            # Plots the PH diagram of the complex created from the Rips filtration
            plt.figure(2)
            plot_diagrams(diagrams, show=True, xy_range = [-0.25,5,-0.25,5])
            
    # Left-to-Right Scanning:
    if sType.lower() == 'lr':
        
        # Tests if the the function is going to do left-to-right
        # slice scanning or not. The scanning interval is changed
        # accordingly.
        if doSlice == False:
            scanStart2 = 1
            scanStop2 = 784
        else:
            scanStart2 = scanStart
            scanStop2 = 784
        
        # Loops through the line with the letter data and if the kth position
        # of the array is a 1, then the coordinates of that 1 is added as a
        # 1x2 array to the letter array.
        for k in range(scanStart2,scanStop2):
            if data[k] == 1:
                row = 28-int((k-1)/28)
                col = (k-1)%28
                letter = np.append(letter,[[row, col]], axis = 0)
        
        # Removes the first entry of letter, as it was a placeholder.
        letter = np.delete(letter, 0, 0)
        
        # Test for the space (or just nothing), otherwise, the function breaks.
        if len(letter) == 0:
            return (0,0)
        
        # Rearranges the xy-coordinates to do left-to-right scanning.
        y_data = np.array([letter[i][0] for i in range(0,len(letter)) for j in range(1,int(scanStop/28)) if letter[i][1] == j])
        x_data = np.array([letter[i][1] for i in range(0,len(letter)) for j in range(1,int(scanStop/28)) if letter[i][1] == j])
        
        # Creates a new array with the the xy-coordinates oriented properly
        # for left-to-right scanning.
        letter2 = np.array([x_data,y_data])
        letter2 = np.transpose(letter2)
        
        # Tests again to make sure there is something in the letter2 array.
        # If there isn't, b0 = b1 = 0.
        if len(letter2) == 0:
            return (0,0)
        
        # Filters the point cloud, using a Rips filtration up to a max dimension of 2.
        diagrams = ripser(letter2, maxdim = 1, metric = 'euclidean',thresh = eps)['dgms']
        
        # If the you want to see the shape of the data and the persistence diagram
        # both are displayed.
        if showPlot == True:
            # Plots the shape of the data
            plt.figure(1)
            plt.plot(x_data, y_data, 'ro', scalex = True, scaley = True)
            axes = plt.gca()
            axes.set_xlim([-1,29])
            axes.set_ylim([-1,29])
        
            # Plots the PH diagram of the complex created from the Rips filtration
            plt.figure(2)
            plot_diagrams(diagrams, show=True, xy_range = [-0.25,5,-0.25,5])
    
    
    # Right-to-Left Scanning:
    if sType.lower() == 'rl':
        
        # Loops through the line with the letter data and if the kth position
        # of the array is a 1, then the coordinates of that 1 is added as a
        # 1x2 array to the letter array.
        for k in range(1,785):
            if data[k] == 1:
                row = 28-int((k-1)/28)
                col = 28-(k-1)%28
                letter = np.append(letter,[[row, col]], axis = 0)
    
        # Removes the first entry of letter, as it was a placeholder.
        letter = np.delete(letter, 0, 0)
        
        # Test for the space (or just nothing), otherwise, the function breaks.
        if len(letter) == 0:
            return (0,0)
        
        # Rearranges the xy-coordinates to do right-to-left scanning.
        y_data = np.array([letter[i][0] for i in range(0,len(letter)) for j in range(1,int(scanStop/10)) if letter[i][1] == j])
        x_data = np.array([letter[i][1] for i in range(0,len(letter)) for j in range(1,int(scanStop/10)) if letter[i][1] == j])
        
        # Creates a new array with the the xy-coordinates oriented properly
        # for Right-to-Left scanning.
        letter2 = np.array([x_data,y_data])
        letter2 = np.transpose(letter2)
        
        # Tests again to make sure there is something in the letter2 array.
        # If there isn't, b0 = b1 = 0.
        if len(letter2) == 0:
            return (0,0)
        
        # Filters the point cloud, using a Rips filtration up to a max dimension of 2.
        diagrams = ripser(letter2, maxdim = 1, metric = 'euclidean',thresh = eps)['dgms']
        
        # If the you want to see the shape of the data and the persistence diagram
        # both are displayed.
        if showPlot == True:
            # Plots the shape of the data
            plt.figure(1)
            plt.plot(x_data, y_data, 'ro', scalex = True, scaley = True)
            axes = plt.gca()
            axes.set_xlim([-1,29])
            axes.set_ylim([-1,29])
        
            # Plots the PH diagram of the complex created from the Rips filtration
            plt.figure(2)
            plot_diagrams(diagrams, show=True, xy_range = [-0.25,5,-0.25,5])
        
    # Calculates the 0-dim betti number.
    betti0 = [1 for x in diagrams[0] if float('inf') in x]
            
    # If there is a 1-dim betti number, it is returned.
    if len(diagrams) == 2:
        
        # Calculates the 1-dim betti number.
        betti1 = [1 for x in diagrams[1] if float('inf') in x]
        
        # Returns the betti numbers.
        return (len(betti0),len(betti1))
    
    # Otherwise b1 = 0.
    else:
        # Returns the betti numbers.
        return(len(betti0),0)

    
# Function to create classification vectors for data sets.
def classificationVector(dataSet, flag):
    '''Inputs: -dataSet: The data set to be analyzed.
               -flag: Flag to control single vector analysis or whole data set analysis.
                       Either 'single' or 'whole'.
    
       Outputs: -dataVect: A matrix containing vectors of weights corresponding
                           to signatures from the data set. (Called dataVect for
                           a single vector, or totalDataVect for the whole set.)
    '''
    
    # List with the scanning types to iterate through.
    scanTypes = ['ud', 'du', 'lr', 'mo-hori', 'dist1', 'ptdim', 'shortlr','ends', 'width']
    
    # Creates classification vectors for each character in the data set.
    if flag.lower() == 'whole':
        
        # List holding numpy arrays of each classification vector.
        totalDataVect = []
    
        j = 0    # loop control variable.
    
        # Iterates through the data set and creates classification vectors
        # for each character.
        while j != len(dataSet):
            
            letter_one_line = dataSet[j,:]    # Loads the data for one character from the data set.
            dataVect = np.array([])           # Creates an empty array to hold the classification features
        
            # Finds the total betti numbers for each data set and places it in the positions 0 and 1
            # of the vector.
            (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 784, eps = 1.42, showPlot = False, doSlice = False)
            dataVect = np.append(dataVect,b0)
            dataVect = np.append(dataVect,b1)
        
            # Iterating through the different scan types and adding to dataVect
            # features depending on the scan type. Each feature is a number between
            # 0 and 1. For betti numbers this is calculated as (100-bn)/100, where
            # n = 0 or 1. For width this is calulated as width/100.
            for sc in scanTypes:
                
                # bottom-to-top scanning is done on the bottom half of the character.
                if sc.lower() == 'du':
                    (b0,b1) = betti_nums(letter_one_line, 'ud', 353, 784, eps = 1.42, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
        
                # Horizontal middle-out scanning on the middle third of the character.
                elif sc.lower() == 'mo-hori':
                    (b0,b1) = betti_nums(letter_one_line, 'ud', 196, 596, eps = 1.42, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
        
                # Width scanning. The scan interval is kept at a fixed [50-i, 50+i] and i is increased until
                # either b1 is greater than or equal to 1, or the entire character is scanned.
                # This width gives the 'time' when the first 1D 'hole' is found. This is calculated as
                # the length of the interval which is 50+i - (50-i) = 2*i. It is then reweighted with division
                # by 100.
                elif sc.lower() == 'width':
                    
                    scanWidth = 0    # Initializes scanWidth to be zero.
                    
                    # Increases the width in increments of 10, with i ranging from 1 to 50.
                    for i in range(1,392,28):
                        
                        # The beginning and end of the scan interval.
                        scStart = 392-i
                        scStop = 392+i
                        
                        # Calculates the betti numbers for the character at the current width.
                        (b0,b1) = betti_nums(letter_one_line, 'lr', scStart, scStop, eps = 1.42, showPlot = False, doSlice = False)

                        # If b1 is greater than or equal to 1, the width calculated as 2*i, and the loop is broken.
                        if b1 >= 1:
                            scanWidth = 2*i
                            break
                    
                    # Appends the width to the classification vector.
                    dataVect = np.append(dataVect,scanWidth/2500)
                    
                    scanWidth = 0    # Resets scanWidth back to zero.
        
                # Up-and-Down scanning, but with the 'infinity' being equal to 1. This connects the skeleton
                # of the character, but without filling in the 'holes' that get filled in a distance of ~1.42.
                elif sc.lower() == 'dist1':
                    (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 784, eps = 1.0, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
        
                # Calculates the number of 'pixels' in the character. This is equivalent to the number of points
                # in the point cloud (i.e. the number of 1's in the data for the character).
                elif sc.lower() == 'ptdim':
                    (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 784, eps = 0, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
            
                # Left-to-Right scanning in a shorter interval than normal. This is done to pick up any 'endpoints'
                # of characters that are missed in the normal left-to-right scan.
                elif sc.lower() == 'shortlr':
                    (b0,b1) = betti_nums(letter_one_line, 'lr', 1, 364, eps = 1.42, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
                    
                # Short scanning Up-and-Down, Down-and-Up, and Left-to-Right. Effectively calculating the ends
                # of the structure if the middle is excised.
                elif sc.lower() == 'ends':
                    (b0u,b1u) = betti_nums(letter_one_line,'ud', 1,168, eps = 1.42, showPlot = False, doSlice = False)
                    (b0d,b1d) = betti_nums(letter_one_line,'ud', 616,784, eps = 1.42, showPlot = False, doSlice = False)
                    (b0l,b1l) = betti_nums(letter_one_line,'lr', 1,225, eps = 1.42, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect, (2500-(b0u+b0d+b0l))/2500)
            
                
                # The remaining scan types (ud and lr) on half of the character.
                else:
                    (b0,b1) = betti_nums(letter_one_line, sc, 1, 674, eps = 1.42, showPlot = False, doSlice = False)
                    dataVect = np.append(dataVect,(2500-b0)/2500)
                    dataVect = np.append(dataVect,(2500-b1)/2500)
        
            # Adds the classifcation vector for the jth character to the total list of vectors.
            totalDataVect.append(dataVect)
            
            # Increments j.
            j += 1
        
        # Returns the total list of classification vectors.
        return totalDataVect
    
    # Creates a classification vector for one character.
    elif flag.lower() == 'single':
        
        letter_one_line = dataSet    # Loads the data for the one character.
        dataVect = np.array([])      # Creates an empty array to hold the classification features
        
        # Finds the total betti numbers for each data set and places it in the positions 0 and 1
        # of the vector.
        (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 2500, eps = 1.42, showPlot = False, doSlice = False)
        dataVect = np.append(dataVect,b0)
        dataVect = np.append(dataVect,b1)
        
        # Iterating through the different scan types and adding to dataVect
        # features depending on the scan type. Each feature is a number between
        # 0 and 1. For betti numbers this is calculated as (100-bn)/100, where
        # n = 0 or 1. For width this is calulated as width/100.
        for sc in scanTypes:
            
            # bottom-to-top scanning is done on the bottom half of the character.
            if sc.lower() == 'du':
                (b0,b1) = betti_nums(letter_one_line, 'ud', 390, 784, eps = 1.42, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
            
            # Horizontal middle-out scanning on the middle third of the character.
            elif sc.lower() == 'mo-hori':
                (b0,b1) = betti_nums(letter_one_line, 'ud', 196, 596, eps = 1.42, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
            
            # Width scanning. The scan interval is kept at a fixed [50-i, 50+i] and i is increased until
            # either b1 is greater than or equal to 1, or the entire character is scanned.
            # This width gives the 'time' when the first 1D 'hole' is found. This is calculated as
            # the width of the interval which is 50+i - (50-i) = 2*i. It is then reweighted with division
            # by 100.
            elif sc.lower() == 'width':
                
                scanWidth = 0    # Initializes scanWidth to be zero.
                
                # Increases the width in increments of 10, with i ranging from 1 to 50.
                for i in range(1,392,28):
                    
                    # The beginning and end of the scan interval.
                    scStart = 392-i
                    scStop = 392+i
                    
                    # Calculates the betti numbers for the character at the current width.
                    (b0,b1) = betti_nums(letter_one_line, 'lr', scStart, scStop, eps = 1.42, showPlot = False, doSlice = False)

                    # If b1 is greater than or equal to 1, the width calculated as 2*i, and the loop is broken.
                    if b1 >= 1:
                        scanWidth = 2*i
                        break
                
                # Appends the width to the classification vector.
                dataVect = np.append(dataVect,scanWidth/2500)
                
                scanWidth = 0    # Resets scanWidth back to zero.
        
            # Up-and-Down scanning, but with the 'infinity' being equal to 1. This connects the skeleton
            # of the character, but without filling in the 'holes' that get filled in a distance of ~1.42.
            elif sc.lower() == 'dist1':
                (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 784, eps = 1.0, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
        
            # Calculates the number of 'pixels' in the character. This is equivalent to the number of points
            # in the point cloud (i.e. the number of 1's in the data for the character).
            elif sc.lower() == 'ptdim':
                (b0,b1) = betti_nums(letter_one_line, 'ud', 1, 784, eps = 0, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
            
            # Left-to-Right scanning in a shorter interval than normal. This is done to pick up any 'endpoints'
            # of characters that are missed in the normal left-to-right scan.
            elif sc.lower() == 'shortlr':
                (b0,b1) = betti_nums(letter_one_line, 'lr', 1, 313, eps = 1.42, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
            
            # Short scanning Up-and-Down, Down-and-Up, and Left-to-Right. Effectively calculating the ends
            # of the structure if the middle is excised.
            elif sc.lower() == 'ends':
                (b0u,b1u) = betti_nums(letter_one_line,'ud', 1,165, eps = 1.42, showPlot = False, doSlice = False)
                (b0d,b1d) = betti_nums(letter_one_line,'ud', 627,784, eps = 1.42, showPlot = False, doSlice = False)
                (b0l,b1l) = betti_nums(letter_one_line,'lr', 1,235, eps = 1.42, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect, (2500-(b0u+b0d+b0l))/2500)
            
            # The remaining scan types (ud and lr) on half of the character.
            else:
                (b0,b1) = betti_nums(letter_one_line, sc, 1, 674, eps = 1.42, showPlot = False, doSlice = False)
                dataVect = np.append(dataVect,(2500-b0)/2500)
                dataVect = np.append(dataVect,(2500-b1)/2500)
        
        return dataVect    # Returns the single classification vector.
    

In [80]:
x_train,x_test,y_train,y_test=train_test_split(X4,Y1,test_size=0.25,random_state=42)

In [81]:
x_tr = classificationVector(x_train,'whole')
x_te=classificationVector(x_test,'whole')

TypeError: list indices must be integers or slices, not tuple

In [ ]:
from keras.layers import Dense # Dense layers are "fully connected" layers
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/
from keras.layers import Dropout, Flatten,Activation
image_size =18 
num_classes = 2 

model = Sequential()

# The input layer requires the special input_shape parameter which should match
# the shape of our training data.
model.add(Dense(units=32, activation='sigmoid', input_shape=(image_size,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dense(units=num_classes, activation='softmax'))


In [ ]:
from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)
model.summary()
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_tr, y_train, batch_size=128, epochs=100,validation_data = (x_te,y_test),
                              verbose = 1)